# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [2]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [3]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [4]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [5]:
from isadream.isadream import modelUtils
from isadream.isadream import io
from isadream.isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [6]:
# A demo json file is provided.
nmr_json_demo = modelUtils.SIPOS_DEMO
demo_base_path = modelUtils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tylerbiggs/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tylerbiggs/git/isadream/isadream/demo_data


---

In [7]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

In [8]:
# node

In [9]:
# help(node)

In [10]:
# import param

In [11]:
# help(param.Parameterized)

In [12]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,sample_node,27 Al ppm,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration
0,a137d5a7-8e5b-4786-85fb-90c1b189b9ae,327bbe01-653a-44da-b96f-1901a7425c5e,e07c8799-b3ab-4bd4-a013-e74d1d8b2e5e,79.90,2.93,2.93,K+,2.93
1,a137d5a7-8e5b-4786-85fb-90c1b189b9ae,327bbe01-653a-44da-b96f-1901a7425c5e,e07c8799-b3ab-4bd4-a013-e74d1d8b2e5e,79.84,4.92,4.92,K+,4.92
2,a137d5a7-8e5b-4786-85fb-90c1b189b9ae,327bbe01-653a-44da-b96f-1901a7425c5e,e07c8799-b3ab-4bd4-a013-e74d1d8b2e5e,79.72,6.85,6.85,K+,6.85
3,a137d5a7-8e5b-4786-85fb-90c1b189b9ae,327bbe01-653a-44da-b96f-1901a7425c5e,e07c8799-b3ab-4bd4-a013-e74d1d8b2e5e,79.66,9.13,9.13,K+,9.13
4,a137d5a7-8e5b-4786-85fb-90c1b189b9ae,327bbe01-653a-44da-b96f-1901a7425c5e,e07c8799-b3ab-4bd4-a013-e74d1d8b2e5e,79.66,10.71,10.71,K+,10.71
0,9919c058-16b6-4e9c-8f5d-3be3c5fbc9e4,40301f5b-ca14-4a65-b7ef-3ad0142fc7cc,d1dfbe71-f550-4ce8-9417-6906a83bde32,79.92,0.66,0.66,Li+,0.66
1,9919c058-16b6-4e9c-8f5d-3be3c5fbc9e4,40301f5b-ca14-4a65-b7ef-3ad0142fc7cc,d1dfbe71-f550-4ce8-9417-6906a83bde32,79.84,1.10,1.10,Li+,1.10
2,9919c058-16b6-4e9c-8f5d-3be3c5fbc9e4,40301f5b-ca14-4a65-b7ef-3ad0142fc7cc,d1dfbe71-f550-4ce8-9417-6906a83bde32,79.66,1.64,1.64,Li+,1.64
3,9919c058-16b6-4e9c-8f5d-3be3c5fbc9e4,40301f5b-ca14-4a65-b7ef-3ad0142fc7cc,d1dfbe71-f550-4ce8-9417-6906a83bde32,79.54,2.14,2.14,Li+,2.14
4,9919c058-16b6-4e9c-8f5d-3be3c5fbc9e4,40301f5b-ca14-4a65-b7ef-3ad0142fc7cc,d1dfbe71-f550-4ce8-9417-6906a83bde32,79.36,2.59,2.59,Li+,2.59


In [13]:
metadata = [md for _, md in node_assays]
metadata
md = collections.ChainMap(*metadata)
# dict(metadata)

In [25]:
md.keys()

KeysView(ChainMap(defaultdict(<class 'tuple'>, {'7bc0a784-2f8f-44ba-9041-a7e0d9812f89': ({'$id': 'https://lampdev02.pnl.gov/bigg006/idreamdrupal/', 'title': 'Sipos 2006, Talanta NMR Figures', 'filename': 'sipos_2006_talanta_nmr_figs.json', 'description': 'Extracted figures.', 'submissionDate': '2018-5-25', 'publicReleaseDate': '2006-03-13', 'experimentSubType': 'Al_NMR'}, [None]), 'e1988d5d-1cc3-4134-baa4-d164caf86b79': (NodeInfo(info={'dataFile': 'sipos_2006_talanta_fig_3_KOH.csv'}, name='NodeInfo00075'), [None]), '2a7e4c6b-864d-411a-8cff-a6b8e465e440': {'sampleName': 'Aluminate Solution'}, '18c201f8-cfd5-4d67-b82e-4d53d24dc1d8': {'sampleName': 'Aluminate Solution'}, '977d884f-a11b-4bb2-b4ef-3b25f966cc40': {'sampleName': 'Potassium Hydroxide'}, '728e2f7c-e1b0-4a80-b189-ec24c63a49bf': {'sampleName': 'Potassium Hydroxide'}, 'e1025e99-bafc-43f9-898a-bab9e40045fa': {'sampleName': 'Potassium Hydroxide'}}), defaultdict(<class 'tuple'>, {'9d992d88-f405-4481-94df-b67991dabf6b': ({'$id': 'http

---